In [ ]:
#QEARL is a python code that takes the orbital parameters of an exoplanet
#and outputs the light curve that planet would generate. It is the quaternion
#version of the EARL Mathematica code previously developed in https://arxiv.org/abs/1802.06805.
#It uses quaternions rather than the Euler angles used in EARL to rotate the spherical harmonics.

import numpy as np
import math as mt
import cmath as cm
import quaternion
import spherical_functions as sf

#This version of the code works for F00 and F10 in all cases. 
#We are currently investigating F11 and higher cases.


# WARNING: Special configs, like edge-on orbit or north/south-pole viewing have not been implemented.

#Define Variables in order wORB, wROT, inclination, obliquity, and solstice phase, time array and maximum l value
Inputs=[0.14,1.22,0.23,0.42,0.67,[.12,.37,0.672],2]
#define variables
wORB =np.array(Inputs[0])
wROT =np.array(Inputs[1])
i =np.array(Inputs[2])
obliquity=np.array(Inputs[3])
solPhase=np.array(Inputs[4])
t=np.array(Inputs[5])
lmax=Inputs[6]

#Here we code the sine and cosine of the time dependent subobserver and substellar point coordinates interms of the orbital parameters

#cosine and sine of the latitude (theta_o) and longitude (phi_o) for the subobserver point in the planet frame
COStheta_o=np.repeat((np.cos(i)*np.cos(obliquity)+np.sin(i)*np.sin(obliquity)*np.cos(solPhase)), len(t))
SINtheta_o=np.sqrt(1-(COStheta_o)**2)
COSphi_o=np.cos(wROT*t)
SINphi_o=-np.sin(wROT*t)

#cosine and sine of the latitude (theta_s) for the substeller point in the planet frame
COStheta_s=np.sin(obliquity)*(np.cos(wORB*t-solPhase))
SINtheta_s=np.sqrt(1-(COStheta_s)**2)

# and longitude (phi_s) for the subosteller point in the planet frame
COSphi_s=(np.cos(wROT*t)*(np.sin(i)*np.cos(wORB*t)-COStheta_o*np.sin(obliquity)\
        *np.cos(wORB*t-solPhase))+ np.sin(wROT*t)*(np.sin(i)*np.sin(wORB*t)*np.cos(obliquity)\
        -np.cos(i)*np.sin(obliquity)*np.sin(wORB*t-solPhase)))/((np.sqrt(1-(np.cos(i)*np.cos(obliquity)\
        +np.sin(i)*np.sin(obliquity)*np.cos(solPhase))**2))
        *(np.sqrt(1-(np.sin(obliquity)**2)*(np.cos(wORB*t-solPhase))**2)))
SINphi_s=(-np.sin(wROT*t)*(np.sin(i)*np.cos(wORB*t)-COStheta_o*np.sin(obliquity)\
        *np.cos(wORB*t-solPhase))+np.cos(wROT*t)*(np.sin(i)*np.sin(wORB*t)\
          *np.cos(obliquity)-np.cos(i)*np.sin(obliquity)*np.sin(wORB*t-solPhase)))\
        /((np.sqrt(1-(np.cos(i)*np.cos(obliquity)+np.sin(i)*np.sin(obliquity)*np.cos(solPhase))**2))\
        *(np.sqrt(1-(np.sin(obliquity)**2)*(np.cos(wORB*t-solPhase))**2)))

#Here we find the quaternion that rotates between the planet and lune frames by buliding the lune frame from the
#sine and cosine of the subobserver and substellar points

#find vector associated with suboberver (nHat_o) and substellar (nHat_s) points
nHat_o=np.stack((SINtheta_o*COSphi_o,SINtheta_o*SINphi_o,COStheta_o),axis=-1)
nHat_s=np.stack((SINtheta_s*COSphi_s,SINtheta_s*SINphi_s,COStheta_s),axis=-1)
luneWidth=np.pi - np.arccos(np.sin(i)*np.cos(wORB*t))

# Lune frame components for each t in the time array
zHat_l=np.divide(np.cross(nHat_o,nHat_s),(np.sin(luneWidth)[:,np.newaxis]))
yHat_l=nHat_s
xHat_l=np.cross(yHat_l,zHat_l)


#create luneFrame in planet frame coordinates
luneFrame=np.stack((xHat_l,yHat_l,zHat_l),axis=-1)
#print(luneFrame)

#calculate quaternion for each lune frame
Q_l=quaternion.from_rotation_matrix(luneFrame)
#print(Q_l[tInd])

#Define Rev and Rodd funcitons
#These functions are only defined for even l and m (Rev) and odd l and m (Rodd)
def Rev(l,m):
    if l==0 and m==0:
        return 2
    else:
        return (2*abs(m)*((mt.factorial(l/2))**2)*mt.factorial((l+m)))\
    /(l*mt.factorial((l-m)/2)*mt.factorial((l+m)/2)*mt.factorial(l+1))

def Rodd(l,m):
    return (-mt.pi*m*mt.factorial(l+m)*mt.factorial(l+1))/\
((l*(2**(2*l+1)))*(((mt.factorial((l+1)/2))**2))*((mt.factorial((l-m)/2)))*(mt.factorial((l+m)/2)))


# Define the functions for the legendere polynomials, PTev and PTodd (even and odd cases for the legendere polynomials)

#even legendre polynomials
def PTev(l,m):
    if m<=l-4:
        return (1/mt.pi)*np.sqrt(((2*l+1)*mt.factorial(l-m))/((4*mt.pi)*mt.factorial(l+m)))\
        *(1/(4*l**2-1))*(((2*l-1)/(2*l+3))*Rev(l+2,m+2)-((4*l+2)/(2*l+3))*Rev(l,m+2)+Rev(l-2,m+2))
    if l-4<m<=l-2:
        return (1/mt.pi)*np.sqrt(((2*l+1)*mt.factorial(l-m))/((4*mt.pi)*mt.factorial(l+m)))\
        *(1/(4*l**2-1))*(((2*l-1)/(2*l+3))*Rev(l+2,m+2)-(((4*l+2)/(2*l+3))*Rev(l,m+2)))
    if l-2<m<=l:
        return (1/mt.pi)*np.sqrt(((2*l+1)*mt.factorial(l-m))/((4*mt.pi)*mt.factorial(l+m)))\
        *(1/(4*l**2-1))*(((2*l-1)/(2*l+3))*Rev(l+2,m+2))
    if m<-l or m>l:
        return 0
#odd legendre polynomials
def PTodd(l,m):
    if m<=l-4:
        return (1/mt.pi)*np.sqrt(((2*l+1)*mt.factorial(l-m))/((4*mt.pi)*mt.factorial(l+m)))\
        *(1/(4*l**2-1))*(((2*l-1)/(2*l+3))*Rodd(l+2,m+2)-((4*l+2)/(2*l+3))*Rodd(l,m+2)+Rodd(l-2,m+2))
    if l-4<m<=l-2:
        return (1/mt.pi)*np.sqrt(((2*l+1)*mt.factorial(l-m))/((4*mt.pi)*mt.factorial(l+m)))\
        *(1/(4*l**2-1))*(((2*l-1)/(2*l+3))*Rodd(l+2,m+2)-(((4*l+2)/(2*l+3))*Rodd(l,m+2)))
    if l-2<m<=l:
        return (1/mt.pi)*np.sqrt(((2*l+1)*mt.factorial(l-m))/((4*mt.pi)*mt.factorial(l+m)))\
        *(1/(4*l**2-1))*(((2*l-1)/(2*l+3))*Rodd(l+2,m+2))
    if m<-l or m>l:
        return 0

#combine PTev and PTodd into one function
def PT(l,m):
    if (l+m) % 2 != 0:
        return 0
    if l % 2==0 and m % 2 ==0:
        return PTev(l,m)
    if l % 2 !=0 and m % 2!=0:
        return PTodd(l,m)

#initalize the LegPol (Legendre Polynomials) array. Will store the values of the legendre polynomials for all l and m between -l and l
LegPol=np.zeros((lmax+1,2*lmax+1))  

#calculate array of Legendre polynomials integrals
#Note that the range ends at lmax+1 here because the end of the range is open.
for l in range(0,lmax+1):    
    for m in range(l,-l-2,-2):
        if m>=0: 
            LegPol[l][m+lmax]=PT(l,m)
        else:
            LegPol[l][m+lmax]=(-1)**m*LegPol[l][-m+lmax]


#define integral in the Lune frame
def Int(m,wORB,i,t):
    w=np.pi - np.arccos(np.sin(i)*np.cos(wORB*t))
    if m==0:
        return (1/2)*(np.sin(w)-w*np.cos(w))
    if m==-2:
        return (1/4)*cm.exp(-1j*w)*(w-np.cos(w)*np.sin(w))
    if m==2:
        return (1/4)*cm.exp(1j*w)*(w-np.cos(w)*np.sin(w))
    else: 
        return (2*1j*np.cos(w)*(1-cm.exp(1j*m*w))-m*np.sin(w)*(1+cm.exp(1j*m*w)))/(m*(m**2-4))


#initialize the PhiArray and create a tIndex to loop over
PhiArray=np.zeros((len(t),2*lmax+1),dtype=np.complex)
tIndex=np.arange(0,len(t))

#calculate PhiArray of integrals at each time
for tv in tIndex:
    for m in range(-lmax,lmax+1):
        PhiArray[tv][m+lmax]=Int(m,wORB,i,t[tv])
        

#initialize the Flm array
Flm=np.zeros((len(tIndex),lmax+1,lmax+1),dtype=np.complex)

#calculate flms
for tInd in tIndex:
    for l in range(0,lmax+1):
        for m in range(0,l+1):  
            mp=np.arange(-l,l+2,2)  #Here I've taken all mp from -l to l in steps of 2.
            mpneg=np.arange(l,-l-2,-2)  # Now, I'll switch to all mp from l to -l in steps of -2.
            ls=np.full(len(mp),[l])
            ms=np.full(len(mp),[-m]) #Here I negate m to agree with EARL code. 
            indices=np.stack((ls,ms,mpneg),axis=-1)
            Flm[tInd][l][m]=np.dot(PhiArray[tInd][mp+lmax]*LegPol[l][mp+lmax],sf.Wigner_D_element(np.x_parity_conjugate(Q_l[tInd]),indices))
print(Flm)